## Managing Analytical Applications Programmatically with APIv2

##### Cloudera Machine Learning exposes a REST API that you can use to perform operations related to projects, jobs, and runs. You can use API commands to integrate CML with third-party workflow tools or to control CML from the command line.

##### This notebook demonstrates how you can deploy the JupyterDash app as a Production Application via CML Analytic Applications. 

##### Together with Notebook 1, you can easily prototype and iterate over different application versions and build CI/CD processes to manage them in a Production Setting

In [1]:
# The CMLAPI package is available by default and does not need to be pip-installed
try:
    import cmlapi
except ModuleNotFoundError:
    import os
    cluster = os.getenv("CDSW_API_URL")[:-1]+"2"
    !pip3 install {cluster}/python.tar.gz
    import cmlapi

from cmlapi.utils import Cursor
import string
import random
import json

try:
    client = cmlapi.default_client()
except ValueError:
    print("Could not create a client.")

##### Using CML APIv2 often requires the current Project ID and a Session ID, both which you can easily obtain as shown below.

In [2]:
import os
project_name = os.environ["CDSW_PROJECT"]
project_id = os.environ["CDSW_PROJECT_ID"]
print("Project Name: ", project_name)
print("Project ID: ", project_id)

Project Name:  jupyterdash-in-cml
Project ID:  rlyy-n655-drcv-i702


In [3]:
session_id = "".join([random.choice(string.ascii_lowercase) for _ in range(6)])
session_id

'ypjngt'

##### The following example deployes script 1B_jupyterdash_workbench_editor.py as the base for a CML Analytic Application. Notice no code changes were made from when we ran the visualization in a CML Session.

## Create Application

#### CreateApplicationRequest
Argument | Type | Description | Notes
------------ | ------------- | ------------- | -------------
**project_id** | **str** | The project's identifier | 
**name** | **str** | Name of the new application. |  
**subdomain** | **str** | The subdomain of the application. The application will be served at the URL http(s)://subdomain.<domain> | 
**description** | **str** | The description of the application. | [optional] 
**script** | **str** | The script to run for the new application. | 
**cpu** | **float** | CPU cores to allocate to application (default 1). | [optional] 
**memory** | **float** | Memory in GB to allocate to application (default 1). | [optional] 
**nvidia_gpu** | **int** | Number of Nvidia GPUs to allocate to this application (default 0). | [optional] 
**environment** | **dict(str, str)** | Default environment variables to include in application. | [optional] 
**kernel** | **str** | Kernel to run the job runs on. Possible values are python3, python2, r, or scala. | [optional] 
**bypass_authentication** | **bool** | Enable unauthenticated access to application | [optional] 


##### Creating an App with APIv2 requires selecting a CML Runtime ID ("image_identifier"). You can easily visualize these with the method below.
##### Any one of the basic Python based CML Runtimes provided out of the box will suffice.

In [4]:
client.list_runtimes()

{'next_page_token': 'eyJwYWdlU2l6ZSI6MTAsIm9mZnNldCI6MTAsInZlcnNpb24iOjEsInNlYXJjaEZpbHRlciI6e30sInNvcnQiOltdfQ==',
 'runtimes': [{'description': 'Includes Apache Zeppelin (v0.10.1) as the '
                              'default editor, along with all base librarires '
                              'and interpreters.',
               'edition': 'Community runtime with Apache Zeppling 0.10.1',
               'editor': 'Zeppelin',
               'full_version': '1.0.1',
               'image_identifier': 'aakulov1/cml-zeppelin-runtime:1.0.1',
               'kernel': 'Python 3.7'},
              {'description': 'JupyterLab with PyCaret',
               'edition': 'CML AutoML with PyCaret',
               'editor': 'JupyterLab',
               'full_version': '1.7',
               'image_identifier': 'docker.io/cvandyke/automl_pycaret:1.7',
               'kernel': 'Python 3.9'},
              {'description': 'JupyterLab with PyCaret',
               'edition': 'CML AutoML with PyCaret',

In [5]:
image_id = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-standard:2021.12.1-b17"

In [6]:
#### Creating application also starts application implicitly.

# This creates a simple application. If using runtimes, the runtime_identifier must be specified.
application_request = cmlapi.CreateApplicationRequest(
    name = "demo_app_"+session_id,
    description = "A sample application to demonstrate CML APIs",
    project_id = project_id,
    subdomain = "demo-"+session_id,
    runtime_identifier = image_id,
    script = "1B_jupyterdash_workbench_editor.py",
)
app = client.create_application(
    project_id = project_id,
    body = application_request
)

##### Navigate to the Applications tab and validate that the JupyterDash visualization has been deployed as a CML App

![title](img/step7.png)

##### With the following methods you can monitor and update your App deployments. These provide a foundation for managing Production Apps in a CI/CD fashion.

## List/Get Applications

Applications can be listed using the same mechanisms (sort, search_filter, page_size, etc.) as the other resources we've seen so far. Applications can be filtered on the following properties:
- creator.email
- creator.name
- creator.username
- description
- full_name
- name
- script
- subdomain
- status
- kernel
- bypass_authentication
- runtime_identifier

Applications can also be sorted on the following properties:
- created_at
- creator.email
- creator.name
- creator.username
- description
- name
- kernel
- script
- updated_at
- status
- runtime_identifier

In [28]:
# You can list applications similarly to other resources.
client.list_applications(project_id = project_id)

{'applications': [{'bypass_authentication': False,
                   'cpu': 1.0,
                   'created_at': datetime.datetime(2022, 10, 18, 22, 33, 51, 332577, tzinfo=tzlocal()),
                   'creator': {'email': 'pauldefusco@cloudera.com',
                               'name': 'Paul de Fusco',
                               'username': 'pauldefusco'},
                   'description': 'A sample application to demonstrate CML '
                                  'APIs',
                   'environment': '{}',
                   'id': 'mbae-ps4j-3kjz-ese9',
                   'kernel': '',
                   'memory': 1.0,
                   'name': 'demo_app_zjaszu',
                   'nvidia_gpu': 0,
                   'running_at': datetime.datetime(2022, 10, 18, 22, 34, 1, 837000, tzinfo=tzlocal()),
                   'runtime_addon_identifiers': ['hadoop-cli-7.2.11-hf4'],
                   'runtime_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime

#### In the below example we redeploy the same App with a different script ("1C_jupyterdash_workbench_editor.py") and a higher CPU and Memory profile.

## Update Application

When updating an application, you can modify the following fields:
- name
- subdomain
- description
- script
- bypass_authentication
- kernel
- cpu
- memory
- nvidia_gpu
- environment

Modifying these fields can be done similarly to how we updated projects and jobs earlier.


In [30]:
update_application_req = cmlapi.Application(
    name = "updated_" + app.name,
    subdomain = "updated-" + app.subdomain,
    description = "updated_" + app.description,
    environment = json.dumps({"UPDATED_ENV": "UPDATED_ENV_VALUE"}),
    script = "1C_jupyterdash_workbench_editor.py",
    cpu = 1,
    memory = 2
)
updated_application = client.update_application(
    update_application_req,
    project_id = project_id,
    application_id = app.id
)
updated_application

{'bypass_authentication': False,
 'cpu': 1.0,
 'created_at': datetime.datetime(2022, 10, 18, 22, 33, 51, 332577, tzinfo=tzlocal()),
 'creator': {'email': 'pauldefusco@cloudera.com',
             'name': 'Paul de Fusco',
             'username': 'pauldefusco'},
 'description': 'updated_A sample application to demonstrate CML APIs',
 'environment': '{"UPDATED_ENV":"UPDATED_ENV_VALUE"}',
 'id': 'mbae-ps4j-3kjz-ese9',
 'kernel': '',
 'memory': 2.0,
 'name': 'updated_demo_app_zjaszu',
 'nvidia_gpu': 0,
 'running_at': datetime.datetime(2022, 10, 18, 22, 49, 28, 710000, tzinfo=tzlocal()),
 'runtime_addon_identifiers': ['hadoop-cli-7.2.11-hf4'],
 'runtime_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-standard:2021.12.1-b17',
 'script': '1C_jupyterdash_workbench_editor.py',
 'starting_at': datetime.datetime(2022, 10, 18, 22, 49, 28, 710000, tzinfo=tzlocal()),
 'status': 'APPLICATION_STOPPING',
 'stopped_at': None,
 'subdomain': 'updated-demo-zjaszu',

##### Navigate back to the Applications tab and validate that the Application Deployment settings have changed

![title](img/step8.png)

![title](img/step9.png)